## Run cleaning algo functions from `Data_Cleansing_Single_file`
### Saksham Gakhar, DA - DKSF

Keep changing the input csv file and look for duplicate and multivalued records, enlist devices that generally misbehave...

In [4]:
import numpy as np 
import pandas as pd
import datetime
%run Data_Cleansing_Single_file.ipynb

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64655 entries, 0 to 64654
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   service_uploaded  64655 non-null  datetime64[ns, UTC]
 1   when_captured     61839 non-null  datetime64[ns, UTC]
 2   device_urn        64655 non-null  object             
 3   device_sn         64655 non-null  object             
 4   device            64655 non-null  int64              
 5   loc_lat           64655 non-null  float64            
 6   loc_lon           64655 non-null  float64            
 7   env_temp          36580 non-null  float64            
 8   env_humid         36580 non-null  float64            
 9   pms_pm01_0        36899 non-null  float64            
 10  pms_pm02_5        36898 non-null  float64            
 11  pms_pm10_0        36897 non-null  float64            
 12  lnd_7318c         55230 non-null  float64            
 13  l

## Run cleaning algo functions

In [6]:
def cleanSolarCastData(dt):
    """
        Function to clean all the data with the helper functions in `Data_Cleansing_Single_file`
        arg: dt: The function returns the cleaned data frame for the YYYY-MM corresponding to "dt"
        return : df: cleaned dataframe
    """

    df = readCSV(dt)
    findBadData(df)

    df = rmInvalidTimeStamps(df)
    print("new df: ", df.shape)

    df = imputeInaccurateRH(df)
    print("new df: ", df.shape)

    dropServiceUploaded(df)
    print("new df after dropping service_uploaded col: ", df.shape)

    rmDuplicates(df)
    print("new df after removing duplicates: ", df.shape)

    merged,num_groups = dataAggWithKey(df)
    print("merged: ", merged.shape)
    print("num_groups : ", num_groups)

    sum1, toDiscard1 = identifyALLNanRecs(merged)
    sum3, toDiscard3 = identifyMultivaluedTimeStamps(merged)
    sum2 = identifyRemainingDupl(merged)
    sum4 = goodTimeStamps(merged)
    print("toDiscard1 shape: ",toDiscard1.shape)
    print("toDiscard3 shape: ",toDiscard3.shape)

    # sanityCheck(): ensure you have all records covered by 1 of the 4 conditions
    assert(num_groups == sum1+sum2+sum3+sum4)

    writeDF(dt, toDiscard3, 'MultivaluedTimeStamps')

    df = filterRows(toDiscard1, toDiscard3, df)
    print("final df shape: ", df.shape)

    ### Now check to make sure no garbage data is left

    badRecordsLeft = findBadData(df)
    badRecordsLeft
    assert(badRecordsLeft.empty)

    return df